In [1475]:
from hmmlearn import hmm
import numpy as np
import warnings
warnings.filterwarnings('ignore')

POLICY_DAYS = 100
COMPONENTS = 8

In [1574]:

listX = []
list_price = []
f=open('./txt-files/ERIC-B.txt', 'r')
f1 = f.readlines()

last_price = None
last_volume = None
for line in f1:
    info = line.split()
    price_str = info[1]
    price_str = price_str.replace(',', '.')
    price = int(price_str) / 100
    volume = int(info[2])
    if last_price == None:
        rate = 0
    else:
        rate = price / last_price - 1
        
    if last_volume == None:
        volume_rate = 0
    else:
        volume_rate = volume / last_volume - 1
    listX.append([rate * 100, volume_rate * 100])
    list_price.append(price)
    last_price = price
    last_volume = volume
    

training_length = int(len(listX) * 7/10)
print(training_length)



877


In [1575]:
# normalization
import statistics as stats

price_rate_list = [item[0] for item in listX]
volume_rate_list = [item[1] for item in listX]

mean = stats.mean(price_rate_list)
stddev = stats.stdev(price_rate_list)
norm_price_rate_list = [(i - mean) / stddev for i in price_rate_list]

mean = stats.mean(volume_rate_list)
stddev = stats.stdev(volume_rate_list)
norm_volume_rate_list = [(i - mean) / stddev for i in volume_rate_list]

listX = []
listX = [[norm_price_rate_list[i], norm_volume_rate_list[i]] for i in range(len(norm_price_rate_list))]
#listX = [[norm_price_rate_list[i]] for i in range(len(norm_price_rate_list))]
print(listX)

[[-0.011773188795660718, -0.17127118406071118], [-1.1531741506894317, -1.0418547146560817], [-0.3785475455067622, -0.611030532084652], [0.3576057630889874, -0.19461914517735818], [0.5884030312770623, -0.22342348814904153], [-1.0336194781463404, 1.2108231841221166], [-0.07901982409791954, -1.2312712397097545], [0.15656102611146877, 2.138770669668639], [0.08890153199073078, -0.2690619345550076], [1.4284436260600792, 0.7918691388942898], [-0.0769552344175914, -0.373018435993199], [-0.14230084853544056, -0.7300866128361386], [0.11908284241516151, -0.6972471047097502], [1.0650800040575041, 2.633244442102815], [0.21201716574897952, -0.719889131806796], [-0.043606280843576865, -0.22857566145827476], [0.9756569671108586, 0.4038070015901874], [-0.2618466138259629, -0.3844704369071349], [-0.3886954526094381, -0.6542249951359839], [0.14642411171011896, -0.04591197725694586], [-0.579556635316849, -0.1866050031174566], [0.30714325759491, -0.1599942850474907], [-0.17025969826032178, -0.2090650011097

In [1576]:
#remodel = hmm.GaussianHMM(n_components=COMPONENTS,  n_iter=1000, random_state = 140,  covariance_type="full")
remodel = hmm.GaussianHMM(n_components=COMPONENTS,  n_iter=1000, random_state = 140)
remodel.fit(listX[:training_length])
Z1=remodel.predict(listX)


In [1577]:
import plotly.plotly as py
import plotly.graph_objs as go


def my_func(idx, cluster_no):
    if(Z1[idx] == cluster_no):
        return list_price[idx]
    else:
        return None


data0_y = [my_func(n, 0) for n in range(len(list_price))]
data1_y = [my_func(n, 1) for n in range(len(list_price))]
data2_y = [my_func(n, 2) for n in range(len(list_price))]
data3_y = [my_func(n, 3) for n in range(len(list_price))]
data4_y = [my_func(n, 4) for n in range(len(list_price))]
data5_y = [my_func(n, 5) for n in range(len(list_price))]

trace0 = go.Scatter(
    x = [n for n in range(len(list_price))],
    y = data0_y,
    connectgaps=False,
    name='0'
)

trace1 = go.Scatter(
    x = [n for n in range(len(list_price))],
    y = data1_y,
    connectgaps=False,
    name='1'
)

trace2 = go.Scatter(
    x = [n for n in range(len(list_price))],
    y = data2_y,
    connectgaps=False,
    name='2'
)

trace3 = go.Scatter(
    x = [n for n in range(len(list_price))],
    y = data3_y,
    connectgaps=False,
    name='3'
)

trace4 = go.Scatter(
    x = [n for n in range(len(list_price))],
    y = data4_y,
    connectgaps=False,
    name='4'
)

trace5 = go.Scatter(
    x = [n for n in range(len(list_price))],
    y = data5_y,
    connectgaps=False,
    name='5'
)

data = [trace0, trace1, trace2, trace3, trace4, trace5]
layout = go.Layout(
    title='test',
    xaxis=dict(
        title='Time(s)',
    ),
    yaxis=dict(
        title='price',
    )
)
fig = go.Figure(data=data, layout=layout)
# py.iplot(data, filename='basic-line')


In [1578]:
from itertools import combinations 

def findCombinations(state_list, num_buy_states, num_sell_states):
    length = len(state_list)
    comb_buy = combinations(state_list, num_buy_states)
    comb_list = []
    for buy_list in list(comb_buy):
        tmp_list = state_list.copy()
        for state in buy_list:
            tmp_list.remove(state)
        comb_sell = combinations(tmp_list, num_sell_states)
        for sell_list in comb_sell:
            #print("buy_list: {}, sell_list: {}".format(buy_list, sell_list))
            comb_list.append([list(buy_list), list(sell_list)])
    return comb_list

def findAllComb(num_components):
    state_list = list(range(num_components))
    length = len(state_list)
    output = []
    for num_buy in range(length):
        for num_sell in range(length):
            comb_list = findCombinations(state_list, num_buy, num_sell)
            output.extend(comb_list)
    return output

In [1579]:
def trade(money, stocks, price, op):
    if op == "buy":
        stocks_inc = int(money / (price))
        stocks += stocks_inc
        money = money - stocks_inc * price
    elif op == "sell":
        money = money + stocks * price
        stocks = 0
    return money, stocks
    
def tradeForToday(money, stocks, state, buy_list, sell_list, price, detail = True):
    op = None
    if state in buy_list and money > price:
        # we buy in
        money, stocks = trade(money, stocks, price, "buy")
        if detail == True:
            print("buy in at day {}: state: {} price: {} current stocks:{}, current money:{}".format(i, state, price, stocks, money))
            op = "buy"
    elif state in sell_list and stocks > 0:
        # we sell out
        money, stocks = trade(money, stocks, price, "sell")
        if detail == True:
            print("sell out at day {}: state: {} price: {} current stocks:{}, current money:{}".format(i, state, price, stocks, money))
            op = "sell"
    return (money, stocks, op)

In [1580]:
def get_verify_price_change_list(price_seq, today, num_days, num_segs):
    price_change_list = []
    start = today - num_days * num_segs
    end = start + num_days
    for i in range(num_segs):
        price_change_list.append(price_seq[end] / price_seq[start] - 1)
        start += num_days
        end += num_days
    return price_change_list

In [1581]:
def get_asset_multi_segs(buy_list, sell_list, price_seq, state_seq, today, num_days, num_segs):
    asset_list = []
    start = today - num_days * num_segs
    end = start + num_days
    for i in range(num_segs):
        asset = check_profit(buy_list, sell_list, price_seq[start:end], 
                                   state_list[start:end])
        asset_list.append(asset) 
        start += num_days
        end += num_days
        
    return asset_list
    

In [1582]:
def get_asset_whole_timeline(buy_list, sell_list, price_seq, state_seq, today):
    total_asset = check_profit(buy_list, sell_list, price_seq[:today], 
                                   state_list[:today])
    return total_asset

    

In [1583]:
def get_price_change_rate_multi_segs(price_seq, today, num_days, num_segs):
    price_change_rate_list = []
    start = today - num_days * num_segs
    end = start + num_days
    for i in range(num_segs):
        price_change_rate = price_seq[end] / price_seq[start] - 1
        price_change_rate_list.append(price_change_rate) 
        start += num_days
        end += num_days
    return price_change_rate_list

In [1584]:
def get_price_change_rate_whole_timeline(price_seq, today):
    return price_seq[today] / price_seq[0] - 1

In [1585]:
def get_weighted_avg(input_list):
    n = len(input_list)
    s = 0
    m = 0
    for i in range(n):
        s += input_list[i] * (i + 1)
        m += (i + 1)
    return s/m

#get_weighted_avg([2,4,6])
    

In [1586]:
def is_all_elem_bigger_than(list_input, num):
    for item in list_input:
        if item <= num:
            return False
    return True

In [1587]:
def get_ema(list_input, length):
    multiplier = (2 / (length + 1))
    output = []
    input_len = len(list_input)
    for i in range(input_len):
        if i == 0:
            output.append(list_input[i])
        else:
            ema = list_input[i] * multiplier + output[-1] * (1 - multiplier)
            output.append(ema)
    return output



In [1588]:
def get_dema(list_input, length):
    list_ema = get_ema(list_input, length)
    print(list_ema)
    list_dema = get_ema(list_ema, length)
    return list_dema
l = get_dema([0.292759999999999, 0.2239749999999998, 0.06788999999999978], 3)
print(l)

[0.292759999999999, 0.2583674999999994, 0.1631287499999996]
[0.292759999999999, 0.2755637499999992, 0.2193462499999994]


In [1589]:
def get_diff_list(input_list):
    length = len(input_list)
    output = []
    for i in range(length):
        if i == 0:
            continue
        output.append(input_list[i] - input_list[i - 1])
    return output


In [1590]:
def get_diff_wavg(input_list):
    list_diff = get_diff_list(input_list)
    list_diff_wavg = get_weighted_avg(list_diff)
    return list_diff_wavg

In [1591]:
# find out the best way 
# TODO: update the model
def check_profit(buy_list, sell_list, price_seq, state_seq):
    #print("start to check: sell_list:{}, buy_list:{}, price change:{}".format(sell_list, buy_list, 
    #                                                                          price_seq[-1] / price_seq[0] - 1 ))
    money = 10000
    stocks = 0

    for i in range(len(state_seq)):
        money, stocks, op = tradeForToday(money, stocks, state_seq[i], buy_list, sell_list, price_seq[i], False)

    total_asset =  stocks * price_seq[-1] + money
    #print("finished: current stocks:{}, current money:{}, total asset: {}".format(stocks, money, total_asset))
    return total_asset

def getProfit(asset, origin_asset):
    return asset / origin_asset - 1

def findPolicy(list_price, state_list, today, policy_days, policy_segs, min_avg_profit):
    buy_list = []
    sell_list = []
    optimized_buy_list = None
    optimized_sell_list = None
    max_asset_wavg = 10000
    max_profit_wavg = 0
    price_rate_change_list = get_price_change_rate_multi_segs(list_price, today, POLICY_DAYS, policy_segs)
    price_rate_change_whole_timeline = get_price_change_rate_whole_timeline(list_price, today)
    price_rate_change_wavg = get_weighted_avg(price_rate_change_list)
    start_day = today - policy_days * policy_segs
    
    price_rate_change_diff_list = get_diff_list(price_rate_change_list)
    price_rate_change_diff_wavg = get_weighted_avg(price_rate_change_diff_list)
    comb = findAllComb(COMPONENTS)
    for i in comb:

        buy_list = i[0]
        sell_list = i[1]
        # find profits for multiple time segments
        # the last element is the profit for entire time.
        # the date is today
        #check_profit_multi_segs(buy_list, sell_list, price_seq, state_seq, today, num_days, num_segs):
        asset_list = get_asset_multi_segs(buy_list, sell_list, list_price, state_list, today, POLICY_DAYS, policy_segs)
        asset_whole_timeline = get_asset_whole_timeline(buy_list, sell_list, list_price, state_list, today)
        profit_whole_timeline = getProfit(asset_whole_timeline, 10000)
        # the profit must beat the market.
        profit_list = [getProfit(asset, 10000) for asset in asset_list]
        asset_wavg = get_weighted_avg(asset_list)
        profit_wavg = get_weighted_avg(profit_list)
        
        profit_diff_list = get_diff_list(profit_list)
        profit_diff_wavg = get_weighted_avg(profit_diff_list)

        #print("comb:{}, profit_list:{}, w_avg{}, w_price{}".format(i, profit_list, weighted_avg, weighted_price_rate_avg))
        if asset_wavg > max_asset_wavg and profit_wavg > min_avg_profit and profit_diff_wavg > 0.1 and price_rate_change_diff_wavg > 0 and is_all_elem_bigger_than(profit_list,0):
            print("comb:{}, profit_list:{}, wavg_asset:{}, wavg_price:{}, price_rate_change_list={},asset_whole_timeline:{}, price_rate_change_whole_timeline:{}, profit_diff_wavg:{}, price_rate_change_diff_wavg={}".format(i, 
                                                                                                  profit_list, 
                                                                                                  asset_wavg, 
                                                                                                  price_rate_change_wavg,
                                                                                                  price_rate_change_list,
                                                                                                  asset_whole_timeline,
                                                                                                  price_rate_change_whole_timeline,
                                                                                                  profit_diff_wavg,
                                                                                                  price_rate_change_diff_wavg))
            optimized_buy_list = buy_list.copy()
            optimized_sell_list = sell_list.copy()
            max_asset_wavg = asset_wavg
            max_profit_wavg = profit_wavg

    return max_profit_wavg, optimized_buy_list, optimized_sell_list

In [1593]:
import sys

money = 10000
stocks = 0
exp_profit = 0
verify_profit = 0
update_now = False
delayed_update = False
optimized_buy_list = None
optimized_sell_list = None
for i in range(training_length + 1, len(list_price)):
    state_list = remodel.predict(listX[:i])
    price_list = list_price[:i]
    if (i - training_length - 1) % 100 == 0 or update_now:
        if stocks > 0:

            delayed_update = True
            continue
        update_now = False
        # rebuilt the HMM
        if i != training_length:
            print("updating HMM model: day: {}".format(i))
            remodel = hmm.GaussianHMM(n_components=COMPONENTS,  n_iter=1000, random_state = 140)
            #remodel = hmm.GaussianHMM(n_components=COMPONENTS,  n_iter=1000, random_state = 140,  covariance_type="full")
            remodel.fit(listX[:i])
            state_list = remodel.predict(listX[:i])
            optimized_buy_list = None
            optimized_sell_list = None
        # we are out of date, find the policy
        

        price_rate_change_list = get_price_change_rate_multi_segs(list_price, i - 1, POLICY_DAYS, 3)
        #print("WARNING: diff_wavg is too low")    
        price_rate_change_diff_list = get_diff_list(price_rate_change_list)
        price_rate_change_diff_wavg = get_weighted_avg(price_rate_change_diff_list)
        if price_rate_change_diff_wavg < 0 or price_rate_change_diff_wavg > 0.4:
            print("price_rate_change_wavg = {} abnormal".format(price_rate_change_diff_wavg))
            continue
            
        
        best_avg_profit, optimized_buy_list, optimized_sell_list = findPolicy(price_list, 
                                                                              state_list, 
                                                                              i - 1, 
                                                                              POLICY_DAYS, 
                                                                              3, 
                                                                              0.15
                                                                              )

        print("optimized buy: {}, optimized sell: {}, best_avg_profit={}".format(optimized_buy_list, 
                                                            optimized_sell_list, 
                                                            best_avg_profit))
            
    if optimized_buy_list == None:
        if stocks > 0:
            money, stocks = trade(money, stocks, list_price[i - 1], "sell")
            print("no idea about the stock, sell it at price: {}!")
            print("sell out at day {}: state: {} price: {} current stocks:{}, current money:{}".format(i,
                                                                                                       state_seq[i - days_back], 
                                                                                                       price_list[i], 
                                                                                                       stocks, 
                                                                                                       money))
            if delayed_update:
                delayed_update = False
                update_now = True
        continue
    
    # if we lost 10%, sell it
    if stocks > 0 and price_list[i - 1] / trade_price < 0.90:
        print("day: {} lost 10%, trade price:{}, sell it at price: {}.".format(i, trade_price, price_list[i - 1]))
        money, stocks = trade(money, stocks, price_list[i - 1], "sell")
        # seems the model is bad
        update_now = True
        continue
    # trace back if we need sell or buy
    for days_back in range(1, 2):
        money, stocks, op = tradeForToday(money, stocks, state_list[i - days_back], 
                                      optimized_buy_list, optimized_sell_list, price_list[i - 1])
        if op == "sell":
            if delayed_update:
                delayed_update = False
                update_now = True
            break
        elif op == "buy":
            trade_price = price_list[i - 1]
            break
            
        
total_asset =  stocks * price_list[-1] + money
print("finished: current stocks:{}, current money:{}, cur price:{} total asset: {}".format(stocks, money, price_list[-1], total_asset))


updating HMM model: day: 878
optimized buy: None, optimized sell: None, best_avg_profit=0
updating HMM model: day: 978
price_rate_change_wavg = -0.08975237360825979 abnormal
updating HMM model: day: 1078
comb:[[0, 5, 7], [1, 6]], profit_list:[0.030084999999999917, 0.009296000000000193, 0.30514800000000086], wavg_asset:11606.868333333337, wavg_price:0.03521879903746217, price_rate_change_list=[0.18285232778597016, -0.18682101513802318, 0.1340341655716164],asset_whole_timeline:8023.460000000004, price_rate_change_whole_timeline:-0.35072100313479626, profit_diff_wavg:0.19030500000000053, price_rate_change_diff_wavg=0.09067900616509528
comb:[[0, 5, 7], [1, 2, 6]], profit_list:[0.05410500000000029, 0.009296000000000193, 0.30537600000000076], wavg_asset:11648.041666666672, wavg_price:0.03521879903746217, price_rate_change_list=[0.18285232778597016, -0.18682101513802318, 0.1340341655716164],asset_whole_timeline:8875.139999999998, price_rate_change_whole_timeline:-0.35072100313479626, profit_d